In [19]:
import pandas as pd
import numpy as np
import Levenshtein
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor


In [20]:
def get_name(x):
    if len(x.split()) == 1:
        return x
    else:
        x = [w for w in x.split() if not any(letter in w for letter in "abcdefghijklmnopqrstuvwxyz")]
        return " ".join(x)

In [21]:
F16 = pd.read_csv(f"data/final_16.csv")
F17 = pd.read_csv(f"data/final_17.csv")
F18 = pd.read_csv(f"data/final_18.csv")


In [22]:
CF16 = pd.read_excel(f"data/classifiche/Classifica16.xlsx")
CF17 = pd.read_excel(f"data/classifiche/Classifica17.xlsx")
CF18 = pd.read_excel(f"data/classifiche/Classifica18.xlsx")
CF16["Pt"] = CF16["Pt"]/(CF16["Pt"].max())
CF17["Pt"] = CF17["Pt"]/(CF17["Pt"].max())
CF18["Pt"] = CF18["Pt"]/(CF18["Pt"].max())
CF16['Squadra'].replace(regex=True,inplace=True,to_replace=r'\xa0',value=r'')
CF17['Squadra'].replace(regex=True,inplace=True,to_replace=r'\xa0',value=r'')
CF18['Squadra'].replace(regex=True,inplace=True,to_replace=r'\xa0',value=r'')


In [23]:
def get_name(x):
    if len(x.split()) == 1:
        return x
    else:
        x = [w for w in x.split() if not any(letter in w for letter in "abcdefghijklmnopqrstuvwxyz")]
        return " ".join(x)

In [24]:
FG16 = pd.read_excel(f"data/Valori FG/ValoriFG16.xlsx")
FG17 = pd.read_excel(f"data/Valori FG/ValoriFG17.xlsx")
FG18 = pd.read_excel(f"data/Valori FG/ValoriFG18.xlsx")

In [25]:
FG16["Nome"] = FG16["Calciatore"].apply(get_name)
FG17["Nome"] = FG17["Calciatore"].apply(get_name)
FG18["Nome"] = FG18["Calciatore"].apply(get_name)


In [26]:
F16['Squadra'] = F16['Squadra'].map(CF16.set_index('Squadra')['Pt'])
F17['Squadra'] = F17['Squadra'].map(CF17.set_index('Squadra')['Pt'])
F18['Squadra'] = F18['Squadra'].map(CF18.set_index('Squadra')['Pt'])
FG16["Quot"]=FG16["Q.Iniz."]
FG16["Quot Fin"]=FG16["Quot."]
FG17["Quot Fin"]=FG17["QuotF"]
FG18["Quot Fin"]=FG18["Quot."]


In [27]:
F16 = pd.merge(FG16,F16,on="Nome")
columns = ["Nome","GP","Quot","Mean","STD","Titolare","Squadra_y","Quot Fin"]
F16 = F16[columns]
F17 = pd.merge(FG17,F17,on="Nome")
columns = ["Nome","GP","Quot","Mean","STD","Titolare","Squadra_y","Quot Fin"]
F17 = F17[columns]
F18 = pd.merge(FG18,F18,on="Nome")
columns = ["Nome","GP","Quot","Mean","STD","Titolare","Squadra_y","Quot Fin"]
F18 = F18[columns]
F18["Mean"].max()

9.287878787878787

In [28]:
F16["Quot"]=F16["Quot"]/(F16["Quot"].max())
F17["Quot"]=F17["Quot"]/(F17["Quot"].max())
F18["Quot"]=F18["Quot"]/(F18["Quot"].max())


In [29]:
def get_name(x):
    if len(x.split()) == 1:
        return x
    else:
        x = [w for w in x.split() if not any(letter in w for letter in "abcdefghijklmnopqrstuvwxyz")]
        return " ".join(x)

In [30]:
tot=[F16,F17,F18]
Final = pd.concat(tot)
Final["Mean"]= Final["Mean"]/Final["Mean"].max()
Final["STD"]= Final["STD"]/Final["STD"].max()



In [31]:
train = Final.sample(frac=0.6, random_state=1)
test = Final.loc[~Final.index.isin(train.index)]
print(train.shape)
print(test.shape)

(742, 8)
(110, 8)


In [32]:
columns = ["Squadra_y","Quot"]
model = LinearRegression()
target = ["Mean","STD"]
model.fit(train[columns], train[target])


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [33]:
predictions = model.predict(test[columns])
mean_squared_error(predictions, test[target])

0.010968696631686526

In [34]:
#Qualcosa ne esce fuori, dopo provo n'attimo a giocarci un po' di più

In [35]:
columns=["Squadra_y","Quot"]
Predictions =model.predict(Final[columns])
Final["Predict"]= Predictions[:,0]
Final["Predict STD"]=Predictions[:,1]

In [36]:
CF19 = pd.read_excel(f"data/Quotazione Squadre 2019.xlsx")
CF19["Pt"] = 1/CF19["Pt"] 
CF19["Pt"] = CF19["Pt"]/CF19["Pt"].max()
CF19

,Squadra,Pt
0,Juventus,1.000000
1,Napoli,0.721429
2,Roma,0.721429
3,Inter,0.721429
4,Milan,0.545946
5,Lazio,0.269333
6,Sampdoria,0.048095
7,Atalanta,0.091818
8,Fiorentina,0.067333
9,Torino,0.067333


In [37]:
F19 = pd.read_csv(f"data/final_19.csv")
#Assegno un valore alle squadre in cui giocano, mi baso sul sito di scommesse
F19['SquadraN'] = F19['Squadra'].map(CF19.set_index('Squadra')['Pt'])
#Droppo le squadre che non c'erano l'anno scorso
F19 = F19.dropna()
F19["Tit"] = F19["Tit"]/F19["Tit"].max()
F19["Quote"]=F19["Qt. A"]
F19["Quote"]=F19["Quote"]/F19["Quote"].max()
columns=["SquadraN","Quote"]
Predictions =model.predict(F19[columns])
F19["Predict"]= Predictions[:,0]
F19["Predict STD"] = Predictions[:,1]


In [38]:
Final["Quot Fin"].max()


59

In [39]:
Final["Quot Fin"] = Final["Quot Fin"]/Final["Quot Fin"].max()

In [40]:
train = Final.sample(frac=0.9, random_state=1)
test= Final.loc[~Final.index.isin(train.index)]

In [41]:
columns = ["Predict","Titolare","Predict STD"]
model = LinearRegression()
target = "Quot Fin"

In [42]:
model.fit(train[columns], train[target])
predictions = model.predict(test[columns])
mean_squared_error(predictions, test[target])


0.002260574862630534

In [43]:
F19["Quote"] = F19["Quote"]/(F19["Quote"].max())
columns = ["Predict","Tit","Predict STD"]
FinVal = model.predict(F19[columns])
F19["FinVal"]= FinVal*59
F19["Quote"]=F19["Quote"]*49
F19["Diff"] = -F19["Quote"]+F19["FinVal"]
print(F19.loc[F19['Nome'] == "VRSALJKO"])

     Unnamed: 0   Id     R      Nome Squadra  Qt. A  Qt. I  Diff.       Tit  \
230         230  523  Dd;E  VRSALJKO   Inter     14     14      0  0.833333   

     SquadraN  Quote   Predict  Predict STD     FinVal      Diff  
230  0.721429   14.0  0.601506     0.306975  14.697486  0.697486  


In [44]:
Gruppi_Mantra = F19.groupby("R")

In [45]:
F19= F19.rename(columns={"R": "R_Mantra"})

In [46]:
Ruo = pd.read_excel(f"data/Statistiche_Fantacalcio_2018-19_Fantagazzetta (1).xlsx")
colonne = ["R","Nome"]
Ruo = Ruo[colonne]

In [47]:
F19 = pd.merge(Ruo,F19,on="Nome")
F19["Predict"]=F19["Predict"]*10.41
F19["Tit"]=F19["Tit"]*0.9
F19.to_excel("data/F19.xlsx")


In [48]:
Gruppi_Classic = F19.groupby("R")
Gruppi_Classic.groups.keys()


dict_keys(['A', 'C', 'D', 'P'])

In [49]:
Attaccanti = Gruppi_Classic.get_group("A")
Centrocampisti = Gruppi_Classic.get_group("C")
Difensori = Gruppi_Classic.get_group("D")
Portieri =Gruppi_Classic.get_group("P")

In [50]:
#Spesa Massima Portieri 8
Portieri["Spesa"]=(Portieri["FinVal"]*8)/Portieri["FinVal"].max()
Portieri["Spesa"] = Portieri["Spesa"].round()
Portieri = Portieri.sort_values(by=["Spesa"],ascending=False)
Difensori["Spesa"]=(Difensori["FinVal"]*8)/Difensori["FinVal"].max()
Difensori["Spesa"] = Difensori["Spesa"].round()
Difensori = Difensori.sort_values(by=["Spesa"],ascending=False)
Centrocampisti["Spesa"]=(Centrocampisti["FinVal"]*30)/Difensori["FinVal"].max()
Centrocampisti["Spesa"] = Centrocampisti["Spesa"].round()
Centrocampisti = Centrocampisti.sort_values(by=["Spesa"],ascending=False)
Attaccanti["Spesa"]=(Attaccanti["FinVal"]*100)/Attaccanti["FinVal"].max()
Attaccanti["Spesa"] = Attaccanti["Spesa"].round()
Attaccanti = Attaccanti.sort_values(by=["Spesa"],ascending=False)

C:\Users\fede9\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\fede9\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\fede9\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [51]:
AllDone = pd.ExcelWriter('AllDone.xlsx', engine='xlsxwriter')
Attaccanti.to_excel(AllDone, sheet_name='Attaccanti')
Centrocampisti.to_excel(AllDone, sheet_name='Centrocampisti')
Difensori.to_excel(AllDone, sheet_name='Difensori')
Portieri.to_excel(AllDone, sheet_name='Portieri')
AllDone.save()


In [52]:
def norm(df):
    df.apply()

In [53]:
Nomi_R=F18[~F18.Nome.isin(F19.Nome)]

In [54]:
Nomi_R17 = F17[~F17.Nome.isin(F19.Nome)]
Nomi_R17 = Nomi_R17[~Nomi_R17.Nome.isin(Nomi_R.Nome)]

In [55]:
Nomi_R16 = F16[~F16.Nome.isin(F19.Nome)]
Nomi_R16 = Nomi_R16[~Nomi_R16.Nome.isin(Nomi_R.Nome)]
Nomi_R16 = Nomi_R16[~Nomi_R16.Nome.isin(Nomi_R17.Nome)]


In [56]:
Nomi_R=F18[F18.Nome.isin(F19.Nome)]
Nomi_R17 = F17[F17.Nome.isin(F19.Nome)]
Nomi_R17 = Nomi_R17[Nomi_R17.Nome.isin(Nomi_R.Nome)]
Nomi_R16 = F16[F16.Nome.isin(F19.Nome)]
Nomi_R16 = Nomi_R16[Nomi_R16.Nome.isin(Nomi_R.Nome)]
Nomi_R16 = Nomi_R16[Nomi_R16.Nome.isin(Nomi_R17.Nome)]

In [57]:
Nomi_R16=Nomi_R16.rename(columns={"GP": "GP16","Quot":"Quot16","Mean":"Mean16",
                                  "STD":"STD16","Titolare":"Titolare16",
                                  "Squadra_y":"Squadra16","Quot Fin":"QuotFin16"})
Nomi_R17=Nomi_R17.rename(columns={"GP": "GP17","Quot":"Quot17","Mean":"Mean17",
                                  "STD":"STD17","Titolare":"Titolare17",
                                  "Squadra_y":"Squadra17","Quot Fin":"QuotFin17"})
Nomi_R18=Nomi_R.rename(columns={"GP": "GP18","Quot":"Quot18","Mean":"Mean18",
                                  "STD":"STD18","Titolare":"Titolare18",
                                  "Squadra_y":"Squadra18","Quot Fin":"QuotFin18"})


In [58]:
Nomi_R = pd.merge(Nomi_R17,Nomi_R18, on= "Nome")
Nomi_R = pd.merge(Nomi_R,Nomi_R16, on= "Nome")
list = ['GP16', 'Quot16', 'Mean16', 'STD16', 'Titolare16', 'Squadra16',
       'QuotFin16', 'GP17', 'Quot17', 'Mean17', 'STD17', 'Titolare17',
       'Squadra17', 'QuotFin17', 'GP18', 'Quot18', 'Mean18', 'STD18',
       'Titolare18', 'Squadra18', 'QuotFin18']
Nomi_Norm = Nomi_R[list].apply(lambda x: x/x.max(), axis=0)


In [59]:
train = Nomi_Norm.sample(frac=0.7, random_state=1).reset_index(drop=True)
test= Nomi_Norm.loc[~Nomi_Norm.index.isin(train.index)]
columns=['GP16', 'Quot16', 'Mean16', 'STD16', 'Titolare16', 'Squadra16',
       'QuotFin16', 'GP17', 'Quot17', 'Mean17', 'STD17', 'Titolare17',
       'Squadra17', 'QuotFin17','Quot18','Titolare18', 'Squadra18']
target=['Mean18', 'STD18',"QuotFin18"]

In [60]:
model.fit(train[columns], train[target])
predictions = model.predict(test[columns])
mean_squared_error(predictions, test[target])

0.004623079493047778

In [61]:
list = ['Nome', 'Tit', 'SquadraN', 'Quote']
MLF19 = F19[list]


In [62]:
Nomi_R19=MLF19.rename(columns={"Quote":"Quot19","Tit":"Titolare19",
                                  "SquadraN":"Squadra19"})
Nomi_R = pd.merge(Nomi_R17,Nomi_R18, on= "Nome")
Nomi_R = pd.merge(Nomi_R,Nomi_R19, on= "Nome")
list = ['GP17', 'Quot17', 'Mean17', 'STD17', 'Titolare17',
       'Squadra17', 'QuotFin17', 'GP18', 'Quot18', 'Mean18', 'STD18',
       'Titolare18', 'Squadra18', 'QuotFin18',"Quot19","Titolare19","Squadra19"]
Nomi_R[list] = Nomi_R[list].apply(lambda x: x/x.max(), axis=0)

In [63]:
Prova = model.predict(Nomi_R[list])
Nomi_R["Mean19"]=Prova[:,0]*9.2878
Nomi_R["STD19"]=Prova[:,1]*4.658712105812988
Nomi_R["QuotFin19"]=Prova[:,2]*45


In [64]:
list =["Nome","Mean19","STD19","QuotFin19"]
Nomifin = Nomi_R[list]

In [65]:
Nomifin.to_excel("data/Returning Players ML.xlsx")


In [66]:
Pr = pd.read_excel(f"data/Statistiche_Fantacalcio_2018-19_Fantagazzetta (1).xlsx")
